# HOMEWORK 5

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

### First Graph

In [42]:
hero_net  = pd.read_csv('hero-network.csv')

In [43]:
hero_net.drop(hero_net[hero_net['hero1'] == hero_net['hero2']].index,inplace=True)
hero_net['hero1'] = hero_net.apply(lambda  row :  row['hero1'].rstrip().rstrip('/'),axis=1)
hero_net['hero2'] = hero_net.apply(lambda  row :  row['hero2'].rstrip().rstrip('/'),axis=1)
hero_net['hero1'] = hero_net.apply(lambda row :  'SPIDER-MAN/PETER PARKER' if(row['hero1'] == 'SPIDER-MAN/PETER PAR') else row['hero1'],axis=1)
hero_net['hero2'] = hero_net.apply(lambda row :  'SPIDER-MAN/PETER PARKER' if(row['hero2'] == 'SPIDER-MAN/PETER PAR') else row['hero2'],axis=1)


In [4]:
hero_net.head()

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL


In [5]:
len(hero_net)

572235

In [44]:
new = hero_net.groupby(['hero1','hero2']).hero2.count().reset_index(name="count") 

In [7]:
new.head()

,hero1,hero2,count
0,24-HOUR MAN/EMMANUEL,"FROST, CARMILLA",1
1,24-HOUR MAN/EMMANUEL,KILLRAVEN/JONATHAN R,1
2,24-HOUR MAN/EMMANUEL,M'SHULLA,1
3,3-D MAN/CHARLES CHAN,ANGEL/WARREN KENNETH,1
4,3-D MAN/CHARLES CHAN,ANT-MAN II/SCOTT HAR,1


In [8]:
new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224169 entries, 0 to 224168
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   hero1   224169 non-null  object
 1   hero2   224169 non-null  object
 2   count   224169 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 5.1+ MB


In [45]:
G1 = nx.Graph()

In [46]:
new.apply(lambda row :  G1.add_edge(row['hero1'],row['hero2'],weight=1/row['count']),axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
224094    None
224095    None
224096    None
224097    None
224098    None
Length: 224099, dtype: object

In [60]:
nx.info(G1)

/tmp/ipykernel_30175/968434106.py:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  nx.info(G1)


'Graph with 6426 nodes and 167207 edges'

### Second Graph

In [11]:
nodes  = pd.read_csv('nodes.csv')

In [12]:
edges  =pd.read_csv('edges.csv')

In [13]:
nodes.head()

,node,type
0,2001 10,comic
1,2001 8,comic
2,2001 9,comic
3,24-HOUR MAN/EMMANUEL,hero
4,3-D MAN/CHARLES CHAN,hero


In [14]:
edges.head()

,hero,comic
0,24-HOUR MAN/EMMANUEL,AA2 35
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5
3,3-D MAN/CHARLES CHAN,COC 1
4,3-D MAN/CHARLES CHAN,H2 251


In [15]:
len(nodes)

19090

In [16]:
len(edges)

96104

In [17]:
G2 = nx.Graph()

In [18]:
edges.apply(lambda row :  G2.add_edge(row['hero'],row['comic']),axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
96099    None
96100    None
96101    None
96102    None
96103    None
Length: 96104, dtype: object

In [48]:
nx.info(G2)

/tmp/ipykernel_30175/1976782796.py:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  nx.info(G2)


'Graph with 19090 nodes and 96104 edges'

# Q5

In [25]:
from networkx.algorithms import community

In [47]:
def graph_n(graph:nx.Graph, N=None):
    """ Function to redute the graph to only the top N heroes """
    if N is None:
        return graph
    edges = pd.read_csv("edges.csv")
    h = edges.groupby("hero").count()   # extract the top n heroes
    
    top_n = h.sort_values(by="comic", ascending=False).head(N).index  # to a list
    n = edges[edges.hero.isin(top_n)]["comic"].to_list() # comic nodes that are linked to those heroes
    new_graph = graph.subgraph(top_n.to_list() + n).copy()    # create a subgraph based on the list
    return new_graph

In [56]:
G = graph_n(G1,1000)

In [57]:
res = community.girvan_newman(G)

In [58]:
comunities = tuple(sorted(c) for c in next(res))